In [36]:
import jqdata               #导入聚宽函数库
from six import StringIO    #使用聚宽readfile函数
import numpy as np
import pandas as pd
import talib

从“研究模块”中提取测试的代码数据
注意：在聚宽中使用“g.变量”表示全局变量，只能策略中使用，研究中无法使用，认为是调用函数

In [37]:
# file_handle = read_file("sample_wait_distill.csv")
# code_wait_distill = pd.read_csv(StringIO(file_handle))
# stock_list_all = list(code_wait_distill['code'])
# print stock_list_all

stock_to_choose = get_fundamentals(query(
        valuation.code, valuation.pe_ratio, 
        valuation.pb_ratio,valuation.market_cap, 
        indicator.eps, indicator.inc_net_profit_annual
    ).filter(
        valuation.pe_ratio < 40,
        valuation.pe_ratio > 5
        # indicator.eps > 0.3,
        # indicator.inc_net_profit_annual > 0.10,
        # indicator.roe > 15
    ).order_by(
        valuation.pb_ratio.asc()
    ).limit(
        50), date=None)

stock_list_all = list(stock_to_choose['code'])
print stock_list_all

['600877.XSHG', '002122.XSHE', '600015.XSHG', '600269.XSHG', '600026.XSHG', '600657.XSHG', '000709.XSHE', '600016.XSHG', '601898.XSHG', '000926.XSHE', '601328.XSHG', '601818.XSHG', '600000.XSHG', '000900.XSHE', '000959.XSHE', '000937.XSHE', '600823.XSHG', '601988.XSHG', '601169.XSHG', '600811.XSHG', '600864.XSHG', '601107.XSHG', '601166.XSHG', '600266.XSHG', '601998.XSHG', '600894.XSHG', '600350.XSHG', '601666.XSHG', '000809.XSHE', '600919.XSHG', '600219.XSHG', '000001.XSHE', '000157.XSHE', '600123.XSHG', '000667.XSHE', '600508.XSHG', '000698.XSHE', '000543.XSHE', '000791.XSHE', '000761.XSHE', '600308.XSHG', '601288.XSHG', '600157.XSHG', '600875.XSHG', '600649.XSHG', '600020.XSHG', '600881.XSHG', '601186.XSHG', '000780.XSHE', '600510.XSHG']


从文件中读取价格数据

In [38]:
stock_name = stock_list_all[5]
stock_price = get_price(stock_name, count = 500, end_date='2017-12-31', frequency='daily', fields=['open', 'close'])

# stock_price[stock_price['close']==0]=np.nan
# stock_price= stock_price.dropna()

close_price = stock_price['close']

print stock_price.head()

            open  close
2015-12-16  6.40   6.40
2015-12-17  6.42   6.66
2015-12-18  6.65   6.63
2015-12-21  6.63   6.78
2015-12-22  6.79   6.82


计算MACD的值

In [39]:
dif, dea, macd = talib.MACD(stock_price['close'].values, fastperiod=12, slowperiod=26, signalperiod=20)

print (dif - dea)[-5:]
print macd[-5:]

[ 0.01547295  0.02043526  0.02129286  0.02207814  0.02925964]
[ 0.01547295  0.02043526  0.02129286  0.02207814  0.02925964]


计算出MACD，signal，hist的值
1、DIF：差离值 DIF = EMA12 - EMA26
2、DEA：根据差离值计算其9日的EMA，即离差平均值，是所求的DEA值。今日DEA = （前一日DEA X 8/10 + 今日DIF X 2/10）
3、MACD：BAR=（DIF-DEA)2，即为MACD柱状图。但是talib中MACD的计算是bar = (dif-dea)1

经过验证，macd = dif - dea

In [40]:
print np.where((macd[:-1] < 0) & (macd[1:] > 0))

(array([111, 157, 191, 255, 275, 415, 486, 490]),)


In [41]:
id_x_gold = np.where((macd[:-1] < 0) & (macd[1:] > 0))[0] + 1
print id_x_gold

[112 158 192 256 276 416 487 491]


需要对位置取第一个值，同时因为MACD已经是加减后的数据，所以要加1

In [42]:
id_x_gold = [elem for elem in id_x_gold if elem < 490 ]  # 防止越界
print id_x_gold


[112, 158, 192, 256, 276, 416, 487]


In [43]:

total_count = 0
win_count = [0 for i in range(6)]    #用数组存储胜利的次数，从两天后开始计算
lose_count = [0 for i in range(6)]   #用数据存储失败的次数，从两天后开始计算


for position in id_x_gold:
    total_count += 1
    
    if close_price[position] < close_price[position + 2]:
        win_count[0] += 1
    else:
        lose_count[0] += 1
        
    if close_price[position] < close_price[position + 3]:
        win_count[1] += 1
    else:
        lose_count[1] += 1
        
    if close_price[position] < close_price[position + 4]:
        win_count[2] += 1
    else:
        lose_count[2] += 1
        
    if close_price[position] < close_price[position + 5]:
        win_count[3] += 1
    else:
        lose_count[3] += 1
        
    if close_price[position] < close_price[position + 6]:
        win_count[4] += 1
    else:
        lose_count[4] += 1
        
    if close_price[position] < close_price[position + 7]:
        win_count[5] += 1
    else:
        lose_count[5] += 1
        
       
print total_count,win_count,lose_count

7 [4, 4, 5, 5, 4, 4] [3, 3, 2, 2, 3, 3]
